This script merges ORF candidates from all experiments into one table in the CSV format.
The resulting table will be stored as {local_folder}/merged_orfs_found_by_any_caller.csv
This is the first step of the ORF score prediction and only need to be executed once when you update the experiment list.

In [1]:
import pandas as pd
import os

from veliadb.base import Session, Assembly

In [2]:
def exon_block_to_veliadb_style(exon_blocks, exon_delimiter = '|'):
    block_sizes = []
    chrom_starts = []
    for e in exon_blocks.split(exon_delimiter):
        e_start, e_end = e.split('-')
        block_sizes.append(str(int(e_end)-int(e_start)))
        chrom_starts.append(str(int(e_start)+1)) # Add 1 to starts to convert to 1-indexed
    return ';'.join(chrom_starts), ';'.join(block_sizes)


def metaorf_call_to_veliadb_hash_string(row, chrom_id_to_assembly_id, bp_df_ids):
    assembly_id = chrom_id_to_assembly_id[row['chrom_id']]
    chrom_starts, block_sizes = exon_block_to_veliadb_style(row['exon_blocks'])
    chrom_start = int(row['orf_start'])+1 # Add 1 to start to convert to 1-indexed
    hash_string = f"{assembly_id}_{chrom_start}_{row['orf_end']}_{row['strand']}_{chrom_starts}_{block_sizes}"
    
    if hash_string in bp_df_ids:
        return hash_string
    else:
        return ""


def add_big_prot_ids(orfs):
    session = Session()
    ucsc_style2assembly_id = {}
    for a in session.query(Assembly).all():
        if a.ucsc_style_name != 'na':
            if "_" in a.ucsc_style_name:
                chrom_name = a.genbank_accession
            else:
                chrom_name = a.ucsc_style_name        
            ucsc_style2assembly_id[chrom_name] = a.id

    bp_df = pd.read_csv('s3://velia-data-dev/VDC_004_annotation/big_prot/v0.9.3/orfset_v0.9.3_full_minlen_15_maxlen_999999999_orfs.csv.gz')
    bp_df_ids = set(bp_df["orf.orf_idx_str"])

    for orf in orfs.values():
        orf["bigprot_id"] = metaorf_call_to_veliadb_hash_string(orf, ucsc_style2assembly_id, bp_df_ids)

In [3]:
def download_feature_set(experiment_name, local_path):
    orf_path = f"s3://velia-piperuns-dev/{experiment_name}/output/{experiment_name}_found_by_any_caller.csv"
    local_path = f"{local_path}/{experiment_name}_found_by_any_caller.csv"
    cmd = f"aws s3 cp {orf_path} {local_path}"
    print(cmd)
    os.system(cmd)
    return local_path

def load_orfs(experiment_name, local_folder, merged_orfs_info, orf_callers):
    data_path = download_feature_set(experiment_name, local_folder)
    if not os.path.exists(data_path):
        return 0
    
    for index, row in pd.read_csv(data_path, sep='\t').iterrows():
        orf_key = (row["chrom_id"], row["orf_start"], row["orf_end"], row["strand"], row["exon_blocks"])
        if orf_key not in merged_orfs_info:
            merged_orfs_info[orf_key] = row
            merged_orfs_info[orf_key] = pd.concat([merged_orfs_info[orf_key],
                                                   pd.Series({"#called": 1,
                                                              "cell_types": experiment_name})])
        else:
            merged_orfs_info[orf_key]["#called"] += 1
            merged_orfs_info[orf_key]["cell_types"] += ("," + experiment_name)
            for orf_caller in orf_callers:
                if (type(row[f"orf_score_{orf_caller}"]) == type("")
                    and (not row[f"orf_score_{orf_caller}"].strip())):
                    continue
                else:
                    if ((type(merged_orfs_info[orf_key][f"orf_score_{orf_caller}"]) == type("") and
                         (not merged_orfs_info[orf_key][f"orf_score_{orf_caller}"].strip()))
                        or (merged_orfs_info[orf_key][f"orf_score_{orf_caller}"] is None)
                        or (float(merged_orfs_info[orf_key][f"orf_score_{orf_caller}"]) <= float(row[f"orf_score_{orf_caller}"]))):
                        merged_orfs_info[orf_key][f"orf_score_{orf_caller}"] = row[f"orf_score_{orf_caller}"]
                        merged_orfs_info[orf_key][f"ORF_type_{orf_caller}"] = row[f"ORF_type_{orf_caller}"]
                        merged_orfs_info[orf_key][f"gene_id_{orf_caller}"] = row[f"gene_id_{orf_caller}"]
                        merged_orfs_info[orf_key][f"transcript_id_{orf_caller}"] = row[f"transcript_id_{orf_caller}"]    
    os.remove(data_path)
    return 1

In [4]:
local_folder= "../data"
orf_callers = ["price", "ribotish", "ribocode"]

merged_orfs_info = {}
count = 0
for experiment_name in open(f"{local_folder}/experiments.txt"):
    experiment_name = experiment_name.strip().strip("/")
    count += load_orfs(experiment_name, local_folder, merged_orfs_info, orf_callers)
print(count)

aws s3 cp s3://velia-piperuns-dev/VPR_orfcalling_20240308012804_SRX663267/output/VPR_orfcalling_20240308012804_SRX663267_found_by_any_caller.csv ../data/VPR_orfcalling_20240308012804_SRX663267_found_by_any_caller.csv
download: s3://velia-piperuns-dev/VPR_orfcalling_20240308012804_SRX663267/output/VPR_orfcalling_20240308012804_SRX663267_found_by_any_caller.csv to ../data/VPR_orfcalling_20240308012804_SRX663267_found_by_any_caller.csv
aws s3 cp s3://velia-piperuns-dev/VPR_orfcalling_20240308012804_SRX663268/output/VPR_orfcalling_20240308012804_SRX663268_found_by_any_caller.csv ../data/VPR_orfcalling_20240308012804_SRX663268_found_by_any_caller.csv
download: s3://velia-piperuns-dev/VPR_orfcalling_20240308012804_SRX663268/output/VPR_orfcalling_20240308012804_SRX663268_found_by_any_caller.csv to ../data/VPR_orfcalling_20240308012804_SRX663268_found_by_any_caller.csv


/tmp/ipykernel_10651/83718651.py:14: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  for index, row in pd.read_csv(data_path, sep='\t').iterrows():


aws s3 cp s3://velia-piperuns-dev/VPR_orfcalling_20240308012804_SRX663271/output/VPR_orfcalling_20240308012804_SRX663271_found_by_any_caller.csv ../data/VPR_orfcalling_20240308012804_SRX663271_found_by_any_caller.csv
download: s3://velia-piperuns-dev/VPR_orfcalling_20240308012804_SRX663271/output/VPR_orfcalling_20240308012804_SRX663271_found_by_any_caller.csv to ../data/VPR_orfcalling_20240308012804_SRX663271_found_by_any_caller.csv


/tmp/ipykernel_10651/83718651.py:14: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  for index, row in pd.read_csv(data_path, sep='\t').iterrows():


aws s3 cp s3://velia-piperuns-dev/VPR_orfcalling_20240308012804_SRX663272/output/VPR_orfcalling_20240308012804_SRX663272_found_by_any_caller.csv ../data/VPR_orfcalling_20240308012804_SRX663272_found_by_any_caller.csv
download: s3://velia-piperuns-dev/VPR_orfcalling_20240308012804_SRX663272/output/VPR_orfcalling_20240308012804_SRX663272_found_by_any_caller.csv to ../data/VPR_orfcalling_20240308012804_SRX663272_found_by_any_caller.csv


...

aws s3 cp s3://velia-piperuns-dev/VPR_orfcalling_20241101000923_SRX24843300/output/VPR_orfcalling_20241101000923_SRX24843300_found_by_any_caller.csv ../data/VPR_orfcalling_20241101000923_SRX24843300_found_by_any_caller.csv
download: s3://velia-piperuns-dev/VPR_orfcalling_20241101000923_SRX24843300/output/VPR_orfcalling_20241101000923_SRX24843300_found_by_any_caller.csv to ../data/VPR_orfcalling_20241101000923_SRX24843300_found_by_any_caller.csv


/tmp/ipykernel_10651/83718651.py:14: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  for index, row in pd.read_csv(data_path, sep='\t').iterrows():


aws s3 cp s3://velia-piperuns-dev/VPR_orfcalling_20241101000923_SRX24843301/output/VPR_orfcalling_20241101000923_SRX24843301_found_by_any_caller.csv ../data/VPR_orfcalling_20241101000923_SRX24843301_found_by_any_caller.csv
download: s3://velia-piperuns-dev/VPR_orfcalling_20241101000923_SRX24843301/output/VPR_orfcalling_20241101000923_SRX24843301_found_by_any_caller.csv to ../data/VPR_orfcalling_20241101000923_SRX24843301_found_by_any_caller.csv


/tmp/ipykernel_10651/83718651.py:14: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  for index, row in pd.read_csv(data_path, sep='\t').iterrows():


aws s3 cp s3://velia-piperuns-dev/VPR_orfcalling_20241101000923_SRX24843302/output/VPR_orfcalling_20241101000923_SRX24843302_found_by_any_caller.csv ../data/VPR_orfcalling_20241101000923_SRX24843302_found_by_any_caller.csv
download: s3://velia-piperuns-dev/VPR_orfcalling_20241101000923_SRX24843302/output/VPR_orfcalling_20241101000923_SRX24843302_found_by_any_caller.csv to ../data/VPR_orfcalling_20241101000923_SRX24843302_found_by_any_caller.csv
aws s3 cp s3://velia-piperuns-dev/VPR_orfcalling_20241101000923_SRX24843303/output/VPR_orfcalling_20241101000923_SRX24843303_found_by_any_caller.csv ../data/VPR_orfcalling_20241101000923_SRX24843303_found_by_any_caller.csv
download: s3://velia-piperuns-dev/VPR_orfcalling_20241101000923_SRX24843303/output/VPR_orfcalling_20241101000923_SRX24843303_found_by_any_caller.csv to ../data/VPR_orfcalling_20241101000923_SRX24843303_found_by_any_caller.csv
190


In [5]:
add_big_prot_ids(merged_orfs_info)

/tmp/ipykernel_10651/1172930309.py:34: DtypeWarning: Columns (4,5,8,14) have mixed types. Specify dtype option on import or set low_memory=False.
  bp_df = pd.read_csv('s3://velia-data-dev/VDC_004_annotation/big_prot/v0.9.3/orfset_v0.9.3_full_minlen_15_maxlen_999999999_orfs.csv.gz')


In [6]:
pd.DataFrame(merged_orfs_info.values()).reset_index().to_csv(
    f"{local_folder}/merged_orfs_found_by_any_caller.csv",
    sep="\t",
    index=False)

In [7]:
pd.DataFrame(merged_orfs_info.values()).shape

(549923, 21)

In [8]:
pd.DataFrame(merged_orfs_info.values()).shape

(549923, 21)